## Day 22: Sporifica Virus

http://adventofcode.com/2017/day/22

### Part 2

The whole solution needs adjusting so I'm starting a new notebook to avoid confusion.

In [5]:
from collections import namedtuple
from pyrsistent import pdeque, pset


State = namedtuple('State', 'weakened infected flagged virus_p virus_d total_infections')


def initialise(data):
    def up(p): return (p[0]-1, p[1])
    def right(p): return (p[0], p[1]+1)
    def down(p): return (p[0]+1, p[1])
    def left(p): return (p[0], p[1]-1)

    weakened = pset()
    infected = pset({(r, c) 
                     for r, row in enumerate(data) 
                     for c, ch in enumerate(row) 
                     if ch == '#'})
    flagged = pset()
    
    virus_position = (len(data) // 2, len(data[0]) // 2)
    virus_direction = pdeque([up, right, down, left])
    
    return State(weakened, infected, flagged, virus_position, virus_direction, 0)


def move(state):
    weakened, infected, flagged, v_p, v_d, infections = state
    
    if v_p in infected:
        # turn right
        v_d = v_d.rotate(-1)
        # flag
        infected = infected.remove(v_p)
        flagged = flagged.add(v_p)
    elif v_p in weakened:
        # infect
        weakened = weakened.remove(v_p)
        infected = infected.add(v_p)
        infections = infections + 1
    elif v_p in flagged:
        # turn around
        v_d = v_d.rotate(2)
        # clean
        flagged = flagged.remove(v_p)
    else: # clean
        # turn left
        v_d = v_d.rotate(1)
        # weaken
        weakened = weakened.add(v_p)
        
    
    # move - left is the leftmost i.e. current direction
    v_p = v_d.left(v_p)
    
    return State(weakened, infected, flagged, v_p, v_d, infections)


def run_n(data, n):
    s = initialise(data)
    
    for _ in range(n):
        s = move(s)
        
    return s

In [10]:
test_data = '''..#
#..
...'''.splitlines()

run_n(test_data, 100)

State(weakened=pset([(-1, 4), (-1, 1), (2, -4), (-2, 0), (-1, 3), (2, 2), (2, -2), (-1, 2), (2, 1), (2, 3), (0, 4), (1, -4), (-2, -1), (0, -2), (1, 0), (4, -1), (4, 0)]), infected=pset([(0, 0), (-1, 0), (3, 0), (2, -1), (3, -1), (1, 1), (-1, -1), (0, -1), (0, 2)]), flagged=pset([(0, 1), (1, 2), (2, -3), (1, 3), (2, 0), (1, -3), (1, -2)]), virus_p=(2, 0), virus_d=pdeque([<function initialise.<locals>.up at 0x7fd8aab50510>, <function initialise.<locals>.right at 0x7fd8aab50598>, <function initialise.<locals>.down at 0x7fd8aab50620>, <function initialise.<locals>.left at 0x7fd8aab506a8>]), total_infections=26)

In [11]:
%time test_final_state = run_n(test_data, 10000000)

CPU times: user 5min 35s, sys: 136 ms, total: 5min 35s
Wall time: 5min 35s


That's quite long but not intolerable. With pypy3

```bash
$ time pypy3 sporifica_virus.py 
2512059

real	1m5.641s
user	1m5.395s
sys	0m0.052s
```

In [ ]:
Pretty straightforward today.